# 微調 Open AI 模型

本筆記本根據 Open AI 官方 [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) 文件的最新指引編寫。

微調能夠透過針對特定應用場景，加入額外的資料與情境，來提升基礎模型的表現。需要注意的是，像 _few shot learning_（少量示例學習）和 _retrieval augmented generation_（檢索增強生成）這類提示工程技巧，可以讓你在預設提示中加入相關資料以提升品質。不過，這些方法會受到目標基礎模型最大 token 視窗大小的限制。

透過微調，我們實際上是用所需的資料重新訓練模型本身（這讓我們能夠使用比最大 token 視窗更多的範例），並部署一個 _自訂_ 版本的模型，推論時就不需要再提供範例。這不僅讓我們在設計提示時更有彈性（token 視窗可以用在其他用途），也有機會降低成本（因為推論時需要傳送給模型的 token 數量減少）。

微調分為四個步驟：
1. 準備訓練資料並上傳。
1. 執行訓練作業，取得微調後的模型。
1. 評估微調模型並反覆調整以提升品質。
1. 當滿意時，將微調模型部署用於推論。

請注意，並非所有基礎模型都支援微調——請參考 [OpenAI 官方文件](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) 以取得最新資訊。你也可以對已經微調過的模型再次進行微調。在本教學中，我們將以 `gpt-35-turbo` 作為微調的目標基礎模型。

---


### 步驟 1.1：準備你的資料集

我們要建立一個聊天機器人，透過押韻詩（limerick）的方式回答元素週期表相關問題，幫助你更了解元素。在這個簡單的教學中，我們只會建立一個資料集，裡面包含幾個範例回應，展示資料的預期格式。實際應用時，你需要建立一個包含更多範例的資料集。如果有現成的開放資料集（適合你的應用領域），你也可以加以利用並重新格式化，以便用於微調。

因為我們專注於 `gpt-35-turbo`，並且只需要單輪回應（chat completion），所以可以用[這個建議格式](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst)來建立範例，符合 OpenAI 聊天回應的需求。如果你預期會有多輪對話內容，則應該使用[多輪範例格式](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst)，這個格式會包含一個 `weight` 參數，用來標示哪些訊息在微調過程中要用（或不用）。

這裡我們會用較簡單的單輪格式來進行教學。資料會採用 [jsonl 格式](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst)，每一行是一筆紀錄，每筆紀錄都是一個 JSON 格式的物件。下面的範例片段展示了兩筆資料——完整的範例集（共 10 筆）請參考 [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl)，這是我們微調教學會用到的資料。**注意：**每一筆紀錄必須寫在同一行，不能像一般 JSON 檔案那樣分行。

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

實際應用時，你會需要更多的範例資料，才能得到較好的結果——這會在回應品質和微調所需時間／成本之間取得平衡。我們這裡用較小的資料集，是為了能快速完成微調，方便說明流程。想看更進階的微調教學，可以參考 [這個 OpenAI Cookbook 範例](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst)。


### 步驟 1.2 上傳你的資料集

請使用 Files API [如這裡所說明](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file) 上傳資料。請注意，為了執行這段程式碼，你必須先完成以下步驟：
 - 已安裝 `openai` Python 套件（請確保使用的版本 >=0.28.0，以支援最新功能）
 - 已將 `OPENAI_API_KEY` 環境變數設為你的 OpenAI API 金鑰
想了解更多，請參考本課程提供的 [設定指南](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst)。

現在，請執行程式碼，將你的本地 JSONL 檔案建立為可上傳的檔案。


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


### 步驟 2.1：使用 SDK 建立微調任務


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


### 步驟 2.2：檢查作業狀態

你可以用 `client.fine_tuning.jobs` API 做以下幾件事：
- `client.fine_tuning.jobs.list(limit=<n>)` - 列出最近 n 筆微調作業
- `client.fine_tuning.jobs.retrieve(<job_id>)` - 取得特定微調作業的詳細資訊
- `client.fine_tuning.jobs.cancel(<job_id>)` - 取消一個微調作業
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - 列出該作業最多 n 筆事件
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

這個流程的第一步是_驗證訓練檔案_，確保資料格式正確。


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


### 步驟 2.3：追蹤事件以監控進度


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### 步驟 2.4：在 OpenAI 儀表板中查看狀態


你也可以透過造訪 OpenAI 官方網站，並在平台的 _Fine-tuning_ 區段中查看狀態。這裡會顯示目前工作的狀態，也能讓你追蹤過去執行紀錄的歷史。在這張截圖中，你可以看到前一次執行失敗，而第二次執行則成功了。補充說明，這是因為第一次執行時使用了一個格式錯誤的 JSON 檔案——修正之後，第二次執行順利完成，模型也成功上線可供使用。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-model-status.563271727bf7bfba7e3f73a201f8712fae3cea1c08f7c7f12ca469c06d234122.tw.png)


您也可以在視覺化儀表板中向下捲動，查看狀態訊息和指標，如下所示：

| 訊息 | 指標 |
|:---|:---|
| ![訊息](../../../../../translated_images/fine-tuned-messages-panel.4ed0c2da5ea1313b3a706a66f66bf5007c379cd9219cfb74cb30c0b04b90c4c8.tw.png) |  ![指標](../../../../../translated_images/fine-tuned-metrics-panel.700d7e4995a652299584ab181536a6cfb67691a897a518b6c7a2aa0a17f1a30d.tw.png)|


### 步驟 3.1：取得 ID 並在程式碼中測試微調後的模型


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


### 步驟 3.2：在 Playground 載入並測試微調後的模型

你現在可以用兩種方式測試微調後的模型。首先，你可以前往 Playground，然後在 Models 下拉選單中，從列表中選擇你剛剛微調完成的模型。另一個方式是使用 Fine-tuning 面板中顯示的 "Playground" 選項（請參考上方截圖），這會開啟一個_比較_視圖，讓你可以同時看到基礎模型和微調模型的回應，方便快速評估。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-compare.56e06f0ad8922016497d39ced3d84ea296eec89073503f2bf346ec9718f913b5.tw.png)

只要填入你訓練資料中使用的 system context，並輸入你的測試問題。你會發現兩邊都會用相同的 context 和問題來更新。執行比較後，你就能看到兩者輸出的差異。_請注意，微調後的模型會依照你範例中提供的格式來產生回應，而基礎模型則只是單純依照 system prompt 回應_。

![Fine-tuning job status](../../../../../translated_images/fine-tuned-playground-launch.5a26495c983c6350c227e05700a47a89002d132949a56fa4ff37f266ebe997b2.tw.png)

你會發現這個比較畫面也會顯示每個模型的 token 數量，以及推論所花費的時間。**這個範例只是為了說明流程而設計的簡化案例，並不代表真實世界的資料集或情境**。你可能會注意到，兩個範例的 token 數量相同（因為 system context 和 user prompt 都一樣），但微調後的模型（自訂模型）推論時間較長。

在真實情境中，你不會用這種玩具範例，而是會用真實資料（例如：客服的產品目錄）來進行微調，這時回應品質的差異會更加明顯。在_這種_情境下，若要讓基礎模型產生同等品質的回應，通常需要更多自訂的 prompt 設計，這會增加 token 的使用量，也可能讓推論所需的處理時間變長。_如果你想實際體驗，可以參考 OpenAI Cookbook 裡的微調範例開始嘗試。_



---

**免責聲明**：  
本文件係使用 AI 翻譯服務 [Co-op Translator](https://github.com/Azure/co-op-translator) 翻譯。雖然我們力求準確，但請注意，自動翻譯可能包含錯誤或不精確之處。原始語言的文件應視為具權威性的來源。對於重要資訊，建議尋求專業人工翻譯。我們對因使用本翻譯而產生的任何誤解或誤釋不承擔任何責任。
